In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
from selenium import webdriver
import os
import urllib

In [2]:
def fetch_image_urls(query: str, max_links_to_fetch: int, sleep_between_interactions: int = 1):

    # build the flipkart query
    search_url = "https://www.flipkart.com/search?q=*{0}"
    
    image_urls = set()
    image_count = 0
    results_start = 0
    search_url = search_url.format(query).replace (" ","%20")
    
    while image_count < max_links_to_fetch-1:
        search_page_html = build_beautifulsoup_object(search_url)
        thumbnail_results = search_page_html.findAll("img", {"class": "_396cs4"})
        image_urls = image_urls.union(extract_image_urls(thumbnail_results))
                
        image_count = len(image_urls)
        # Extract more images to meet the count
        if image_count >= max_links_to_fetch:
            print(f"Found: {len(image_urls)} image links, done!")
            break
        else:
            print("Found:", image_count, "image links, looking for more ...")
            # Check if more search result pages available
            if search_page_html.find("a", {"class": "_1LKTO3"}):
                if search_page_html.findAll("a", {"class": "_1LKTO3"})[-1]['href'].find('http') > 0:
                    search_url = search_page_html.findAll("a", {"class": "_1LKTO3"})[-1]['href']
                else:
                    search_url = "https://www.flipkart.com" + search_page_html.findAll("a", {"class": "_1LKTO3"})[-1]['href']
            else:
                print("Found:", image_count, "image links, no more available.")
                break;

    return image_urls

In [3]:
def extract_image_urls(thumbnail_results: list):
    image_urls = set()
    try:
        for img in thumbnail_results:
            # extract image urls
            if img['src'] and 'http' in img['src']:
                image_urls.add(img['src'])
                
    except Exception as e:
        print(f"ERROR - Extracting the image URLs - {e}")
    return image_urls

In [4]:
def build_beautifulsoup_object(url: str):
    try:
        # get all image thumbnail results
        uClient = uReq(url)
        flipkartPage = uClient.read()
        uClient.close()
        search_page_html = bs(flipkartPage, "html.parser")
        
    except Exception as e:
        print(f"ERROR - building the Beautiful Soup object {url} - {e}")
        
    return search_page_html                               

In [5]:
def download_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [6]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        download_image(target_folder, elem, counter)
        counter += 1

In [8]:
DRIVER_PATH = r'./chromedriver.exe'
search_term = 'Coffee Machine'
# num of images you can pass it from here  by default it's 10 if you are not passing
#number_images = 50
search_and_download(search_term=search_term, driver_path=DRIVER_PATH, number_images=50)

C:\Users\ashwin.prabhu\AppData\Local\Temp\ipykernel_41912\2535471838.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:


Found: 40 image links, looking for more ...
Found: 69 image links, done!
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/kybvo280/coffee-maker/n/3/j/coffee-tea-maker-electric-kettle-2-l-multipurpose-large-size-tea-original-imagakxzgnffrbd2.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/khxqt8w0-0/electric-kettle/b/c/3/scarlet-electric-kettle-2-litre-elegant-design-for-hot-water-tea-original-imafxuas3yzz8yfg.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/kolsscw0/mug/o/s/d/steel-travel-outdoor-electric-mug-12-v-car-charging-electric-original-imag3yrna4cbpwh2.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/jl9rf680/coffee-maker/x/v/w/instacuppa-french-press-coffee-maker-600-ml-4-part-superior-original-imaf8fkmt9ukytwz.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/cof

SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/kvfkivk0/coffee-maker/i/x/d/coffee-beater-ii-froth-maker-ii-smart-stirrer-ii-modern-style-original-imag8bxz2qkudzgs.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/kzblocw0/coffee-maker/s/n/r/12345-laxmistore-original-imagbd2gzmcd9sqz.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/ku79vgw0/coffee-maker/t/v/n/hd7432-20-philips-original-imag7dzytzyyh44v.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/l0sgyvk0/coffee-maker/c/l/a/grand-grand-coffee-grinder-stainless-steel-electric-capacity-60-original-imagcg4xvkgsxtp2.jpeg?q=70 - as ./images\coffee_machine
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/k94j24w0/coffee-maker/u/8/m/instacuppa-manual-coffee-grinder-with-adjustable-setting-conical-original-imafqzned8xhbxkj.jpeg?q=70 - as ./images\coffee_machine
SUCC